In [ ]:
%pip install transformers
%pip install datasets
%pip install peft
%pip install bitsandbytes

  Using cached peft-0.11.1-py3-none-any.whl (251 kB)
  Using cached accelerate-0.32.1-py3-none-any.whl (314 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia

In [ ]:
import torch
from torch.utils.data import  DataLoader,TensorDataset
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BlenderbotForCausalLM,AutoTokenizer,AdamW
from datasets import Dataset,DatasetDict
from peft import LoraConfig,TaskType,get_peft_model
from google.colab import drive
import bitsandbytes as bnb
from sklearn.metrics import precision_score, recall_score
from nltk.translate.bleu_score import corpus_bleu


In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open('/content/drive/MyDrive/Bbot/osBookDoc.txt','r',encoding='utf-8', errors='ignore') as p:
    s=p.readlines()

In [ ]:
s

['\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 'OPERATING SYSTEM CONCEPTS\n',
 'NINTH EDITION\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 'OPERATING SYSTEM CONCEPTS\n',
 'ABRAHAM SILBERSCHATZ\n',
 'Yale University\n',
 '\n',
 'PETER BAER GALVIN\n',
 '\n',
 'Vice President and Executive Publisher\tDon Fowley\n',
 'Executive Editor\tBeth Lang Golub\n',
 'Editorial Assistant\tKatherine Willis\n',
 'Executive Marketing Manager\tChristopher Ruel\n',
 'Senior Production Editor\tKen Santor\n',
 'Cover and title page illustrations\tSusan Cyr\n',
 'Cover Designer\tMadelyn Lesure\n',
 'Text Designer\tJudy Allan\n',
 '\n',
 '\n',
 '\n',
 '\n',
 'This book was set in Palatino by the author using LaTeX and printed and bound by Courier- Kendallville. The cover was printed by Courier.\n',
 '\n',
 'Copyright  2013, 2012, 2008 John Wiley & Sons, Inc. All rights reserved.\n',
 '\n',
 '\n',
 'No part of this publication may be reproduced, stored in a retrieval system or transmi

In [ ]:
s=s[706:]

In [ ]:
len(s)

20409

In [ ]:
for i in range(len(s)):
    try:
        s.remove('\n')
    except:
        continue

In [ ]:
len(s)

12045

In [ ]:
s

['   Internally, operating systems vary greatly in their makeup, since they are organized along many different lines. The design of a new operating system is a major task. It is important that the goals of the system be well defined before the design begins. These goals form the basis for choices among various algorithms and strategies.\n',
 '   Because an operating system is large and complex, it must be created piece by piece. Each of these pieces should be a well-delineated portion of the system, with carefully defined inputs, outputs, and functions.\n',
 'Introduction\n',
 'An operating system is a program that manages a computers hardware. It also provides a basis for application programs and acts as an intermediary between the computer user and the computer hardware. An amazing aspect of operating systems is how they vary in accomplishing these tasks. Mainframe operating systems are designed primarily to optimize utilization of hardware. Personal computer (PC) operating systems s

In [ ]:
dataDict={'context':s}

In [ ]:
data=pd.DataFrame(dataDict)

In [ ]:
data

,context
0,"Internally, operating systems vary greatly ..."
1,Because an operating system is large and co...
2,Introduction\n
3,An operating system is a program that manages ...
4,Before we can explore the details of compu...
...,...
12040,X\n
12041,"x86-64 architecture, 387\n"
12042,"Xen, 714\n"
12043,Z\n


In [ ]:
train,val=train_test_split(data,random_state=42)

In [ ]:
device=torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
def replace_linear_with_quantized(model):
    for name, module in model.named_children():
        if isinstance(module, torch.nn.Linear):
            quantized_layer = bnb.nn.Linear8bitLt(
                module.in_features, module.out_features, bias=module.bias is not None
            )
            setattr(model, name, quantized_layer)
        else:
            replace_linear_with_quantized(module)


In [ ]:
model=BlenderbotForCausalLM.from_pretrained('facebook/blenderbot-400M-distill').to(device)
tokenizer=AutoTokenizer.from_pretrained('facebook/blenderbot-400M-distill')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/730M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/347 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/127k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/62.9k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/16.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/310k [00:00<?, ?B/s]

In [ ]:

replace_linear_with_quantized(model)


In [ ]:
def prep(examples):
    inputs=[]
    mask=[]
    labels=[]
    for prompt in examples['context']:

        input=tokenizer(str(prompt),max_length=256,truncation=True,padding='max_length',return_tensors='pt')
        inputs.append(input['input_ids'])
        mask.append(input['attention_mask'])
        labels.append(input['input_ids'])
    return inputs,mask,labels

In [ ]:
trainInputs, trainMasks, trainLabels =prep(train)
valinp,valmask,vallabel=prep(val)

In [ ]:
trainInputs = torch.nn.utils.rnn.pad_sequence([torch.tensor(seq, dtype=torch.long) for seq in trainInputs], batch_first=True, padding_value=tokenizer.pad_token_id).to(device)
trainMasks = torch.nn.utils.rnn.pad_sequence([torch.tensor(seq, dtype=torch.long) for seq in trainMasks], batch_first=True, padding_value=0).to(device)
trainLabels = torch.nn.utils.rnn.pad_sequence([torch.tensor(seq, dtype=torch.long) for seq in trainLabels], batch_first=True, padding_value=tokenizer.pad_token_id).to(device)
valinp      =   torch.nn.utils.rnn.pad_sequence([torch.tensor(seq, dtype=torch.long) for seq in valinp], batch_first=True, padding_value=tokenizer.pad_token_id).to(device)
valmask     =   torch.nn.utils.rnn.pad_sequence([torch.tensor(seq, dtype=torch.long) for seq in valmask], batch_first=True, padding_value=tokenizer.pad_token_id).to(device)
vallabel    =   torch.nn.utils.rnn.pad_sequence([torch.tensor(seq, dtype=torch.long) for seq in vallabel], batch_first=True, padding_value=tokenizer.pad_token_id).to(device)

<ipython-input-22-8b6bdeb3a8af>:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  trainInputs = torch.nn.utils.rnn.pad_sequence([torch.tensor(seq, dtype=torch.long) for seq in trainInputs], batch_first=True, padding_value=tokenizer.pad_token_id).to(device)
<ipython-input-22-8b6bdeb3a8af>:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  trainMasks = torch.nn.utils.rnn.pad_sequence([torch.tensor(seq, dtype=torch.long) for seq in trainMasks], batch_first=True, padding_value=0).to(device)
<ipython-input-22-8b6bdeb3a8af>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torc

In [ ]:
trainData=TensorDataset(trainInputs,trainMasks,trainLabels)
trainData=DataLoader(trainData,shuffle=True,batch_size=4)

In [ ]:
valDataset=TensorDataset(valinp,valmask,vallabel)
valDataloader=DataLoader(valDataset,batch_size=4,shuffle=True)

In [ ]:
model.config

BlenderbotConfig {
  "_name_or_path": "facebook/blenderbot-400M-distill",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": true,
  "architectures": [
    "BlenderbotForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1280,
  "decoder_attention_heads": 32,
  "decoder_ffn_dim": 5120,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 1,
  "do_blenderbot_90_layernorm": true,
  "dropout": 0.1,
  "encoder_attention_heads": 32,
  "encoder_ffn_dim": 5120,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 2,
  "encoder_no_repeat_ngram_size": 3,
  "eos_token_id": 2,
  "extra_layer_norm": false,
  "extra_pos_embeddings": 0,
  "force_bos_token_to_be_generated": false,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
 

In [ ]:
lora_config = LoraConfig(
    r=64,                                # Adapter size or other fine-tuning parameter
    lora_alpha=32,                       # Adapter initialization strength
    target_modules=[
    "model.shared",                       # Shared modules across encoder and decoder
    "model.encoder.layers",               # Encoder layers for processing input sequences
    "model.encoder.self_attn.q_proj",     # Attention projection for encoder self-attention query
    "model.encoder.self_attn.v_proj",     # Attention projection for encoder self-attention value
    "model.decoder.layers.0.self_attn.q_proj", # Example of accessing a specific Linear layer
    "model.decoder.layers.0.self_attn.v_proj", # Example of accessing another specific Linear layer
],
    lora_dropout=0.05,                   # Dropout rate for LORA
    bias="none",                         # Bias setting (e.g., none, fixed, adapt)
    task_type=TaskType.CAUSAL_LM         # Task type for causal language modeling (adjust if necessary)
)


In [ ]:
model=get_peft_model(model,lora_config)

In [ ]:
model.print_trainable_parameters()

trainable params: 327,680 || all params: 335,859,200 || trainable%: 0.0976


In [ ]:
batchSize=4
learningRate=2e-5
epochs=400

In [ ]:
optimizer  = AdamW(model.parameters(),lr=learningRate)
criterion  = torch.nn.CrossEntropyLoss()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
%pip install tqdm

In [ ]:
import tqdm

In [ ]:
def train(model,optimizer,criterion,DataLoader):
    from tqdm.auto import tqdm
    model.train()
    tloss=0
    tcorrect=0 # Initialize a variable to store the number of correct predictions
    total_samples = 0 # Initialize a variable to store the total number of samples
    for batch,(inputs,mask,lab)in enumerate(DataLoader):
        inputs=inputs.to(device)
        mask=mask.to(device)
        label=lab.to(device)
        optimizer.zero_grad()
        outputs=model(input_ids=inputs,attention_mask=mask,labels=label)
        loss=outputs.loss
        loss.backward()
        optimizer.step()
        tloss+=loss.item()
        preds = torch.argmax(outputs.logits, dim=-1)
        
        # Update the number of correct predictions and total samples
        tcorrect += (preds == label).sum().item()
        total_samples += label.size(0)
        
        torch.cuda.empty_cache()
    accuracy = tcorrect / total_samples


    return tloss/len(DataLoader),accuracy 

In [ ]:
def validate(model,dataloader,criterion,device):
    model.eval()
    total_loss=0
    tcorrect=0 # Initialize a variable to store the number of correct predictions
    total_samples = 0 # Initialize a variable to store the total number of samples
    all_labels = []
    all_preds = []
    all_references = []
    all_hypotheses = []
    with torch.no_grad():
        for batch,(inputs,mask,lab)in enumerate(dataloader):
            inputs=inputs.to(device)
            mask=mask.to(device)
            label=lab.to(device)
            outputs=model(input_ids=inputs,attention_mask=mask,labels=label)
            loss = outputs.loss
            total_loss += loss.item()
            preds = torch.argmax(outputs.logits, dim=-1)
            tcorrect += (preds == label).sum().item()
            total_samples += label.size(0)
        
            
            all_labels.extend(label.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
            for ref, hyp in zip(label, preds):
                all_references.append([tokenizer.decode(ref, skip_special_tokens=True).split()])
                all_hypotheses.append(tokenizer.decode(hyp, skip_special_tokens=True).split())
    
    avgloss = total_loss / len(dataloader)
    
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    bleu_score = corpus_bleu(all_references, all_hypotheses)
    accuracy = tcorrect / total_samples

    avgloss=total_loss/len(dataloader)
    return avgloss,accuracy, precision, recall, bleu_score



In [ ]:
type(trainData)

torch.utils.data.dataloader.DataLoader

In [ ]:
model.to(device)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): BlenderbotForCausalLM(
      (model): BlenderbotDecoderWrapper(
        (decoder): BlenderbotDecoder(
          (embed_tokens): BlenderbotScaledWordEmbedding(8008, 1280, padding_idx=0)
          (embed_positions): BlenderbotLearnedPositionalEmbedding(128, 1280)
          (layers): ModuleList(
            (0): BlenderbotDecoderLayer(
              (self_attn): BlenderbotAttention(
                (k_proj): Linear8bitLt(in_features=1280, out_features=1280, bias=True)
                (v_proj): lora.Linear(
                  (base_layer): Linear8bitLt(in_features=1280, out_features=1280, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.05, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=1280, out_features=64, bias=False)
                  )
                  (lora_B): ModuleDict(
                    (de

In [ ]:
for epoch in tqdm.trange(epochs,desc="Epoch(s)"):
    train_loss,acc=train(model,optimizer,criterion,trainData)
    valloss,valacc, valprecision, valrecall, valbleu_score=validate(model,valDataloader,criterion,device)
    dataDict={
        "Epoch": epoch+1,
        "Train Loss": train_loss,
        "Train Accuracy":acc,
        "Validation Loss": valloss,
        "Validation Accuracy":valacc,
        "Precision": valprecision, 
        "Recall":valrecall, 
        "Bleu Score",valbleu_score,
    }
    print(dataDict)
    model.save_pretrained("tunedModels/trainedSmallBlenderBotLoRA - Epoch {}".format(epoch))


Epoch(s):   0%|          | 0/400 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


{'Epoch': 1, 'Train Loss': 2.471923379836857, 'Train Accuracy': 0, 'Validation Accuracy': 1.8077404703870712}


Epoch(s):   0%|          | 1/400 [08:12<54:34:49, 492.46s/it]

{'Epoch': 2, 'Train Loss': 1.7976256502036132, 'Train Accuracy': 0, 'Validation Accuracy': 1.7905873061809565}


Epoch(s):   0%|          | 2/400 [16:22<54:18:14, 491.19s/it]

...

Epoch(s):   1%|          | 3/400 [24:32<54:04:32, 490.36s/it]

{'Epoch': 4, 'Train Loss': 1.733291982443473, 'Train Accuracy': 0, 'Validation Accuracy': 1.7088034464266513}


Epoch(s):   1%|          | 4/400 [32:41<53:52:45, 489.81s/it]

{'Epoch': 5, 'Train Loss': 1.6617378731608232, 'Train Accuracy': 0, 'Validation Accuracy': 1.6026119746040222}


Epoch(s):   1%|▏         | 5/400 [40:48<53:38:18, 488.86s/it]

In [ ]:
def zip_directory(directory_path, zip_name):
    with zipfile.ZipFile(zip_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for folder_name, subfolders, filenames in os.walk(directory_path):
            for filename in filenames:
                file_path = os.path.join(folder_name, filename)
                zipf.write(file_path, os.path.relpath(file_path, directory_path))


In [ ]:
save_directory = "tunedModels/trainedSmallBlenderBotLoRA_Epoch_{}".format(epoch)
zip_name = f'/content/drive/MyDrive/Bbot/{save_directory}.zip'

In [ ]:
zip_directory(save_directory, zip_name)